In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = '/content/drive/My Drive/Colab Notebooks/Research Techniques II/merged_dataset_v2.csv'

In [ ]:
df.to_csv('/content/drive/My Drive/Colab Notebooks/Research Techniques II/merged_dataset_v2.csv')

In [3]:
import pandas as pd

df = pd.read_csv(file_path, index_col = 0)
df.head()

,Unnamed: 0,text,labels
0,0,Bu tətbiq niyə nömrəyə kod göndərmirr,0
1,1,Biz evin qida məhsullarini bravodan alırıq çox...,1
2,2,Pisdir çox exlaqsizliq bas alıb gedir,0
3,3,Qulag asmiram amma merifetli qardasdi Allah ca...,1
4,4,m10 kimi rahat gözəl tətbiq yoxdur,1


In [ ]:
print(df.columns)

Index(['Unnamed: 0', 'text', 'labels'], dtype='object')


In [ ]:
df['labels'].value_counts()

,count
labels,
1,32832
0,22841


In [ ]:
len(df)

55673

In [5]:
from sentence_transformers import SentenceTransformer

# LaBSE modelini yükle
model = SentenceTransformer('sentence-transformers/LaBSE')

# Temizlenmiş metinleri al
texts = df['text'].tolist()

# Embedding çıkar (batch size isteğe göre artırılabilir)
embeddings = model.encode(texts, batch_size=64, show_progress_bar=True)

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

Batches:   0%|          | 0/870 [00:00<?, ?it/s]

In [6]:
import numpy as np

# NumPy array'e dönüştürmek istiyorsan:
embeddings = np.array(embeddings)

# Şekli kontrol et (satır sayısı, embedding boyutu)
print(embeddings.shape)

(55673, 768)


In [7]:
X_seq = np.expand_dims(embeddings, axis=1)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_seq,
    df['labels'].values,
    test_size=0.2,
    stratify=df['labels'].values  # mutlaka kalmalı
)

In [44]:
import tensorflow as tf
from keras.layers import Layer

class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='att_weight', shape=(input_shape[-1], 1),
                                 initializer='random_normal', trainable=True)
        self.b = self.add_weight(name='att_bias', shape=(input_shape[1], 1),
                                 initializer='zeros', trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x):
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

In [48]:
from keras.models import Sequential
from keras.layers import Bidirectional, GRU, Dense, Dropout, BatchNormalization, Input
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

model = Sequential()
model.add(Input(shape=(1, X_seq.shape[2])))  # (seq_len=1, embedding_dim=768)

# units1 = 64 ve tek GRU katmanı, return_sequences=False çünkü başka GRU yok
model.add(Bidirectional(GRU(64)))
model.add(BatchNormalization())
model.add(Dropout(0.4))

# Dense katmanı
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    verbose=1
)

model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/10
1392/1392 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.7842 - loss: 0.4549 - val_accuracy: 0.8171 - val_loss: 0.3924 - learning_rate: 0.0010
Epoch 2/10
1392/1392 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy: 0.8152 - loss: 0.4071 - val_accuracy: 0.8203 - val_loss: 0.3854 - learning_rate: 0.0010
Epoch 3/10
1392/1392 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.8174 - loss: 0.3981 - val_accuracy: 0.8194 - val_loss: 0.3801 - learning_rate: 0.0010
Epoch 4/10
1392/1392 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.8248 - loss: 0.3857 - val_accuracy: 0.8244 - val_loss: 0.3788 - learning_rate: 0.0010
Epoch 5/10
1392/1392 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.8274 - loss: 0.3853 - val_accuracy: 0.8225 - val_loss: 0.3770 - learning_rate: 0.0010
Epoch 6/10
1392/1392 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.8255 - loss: 0.3798 - val_accuracy: 0.8253 - val_loss: 0.3768 - learning_rate: 0.0010
Epoch 7/10
1392/1392 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy: 0.8

In [49]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

348/348 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
              precision    recall  f1-score   support

           0       0.76      0.84      0.80      4568
           1       0.88      0.82      0.85      6567

    accuracy                           0.83     11135
   macro avg       0.82      0.83      0.82     11135
weighted avg       0.83      0.83      0.83     11135



In [53]:
model.save('/content/drive/MyDrive/Colab Notebooks/Research Techniques II/my_model.keras')

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/Colab Notebooks/Research Techniques II/labse_bigru/my_model.keras')

In [25]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 12.1 MB/s eta 0:00:00


In [41]:
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import Bidirectional, GRU, Dense, Dropout, BatchNormalization, Input

def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(1, X_seq.shape[2])))

    # İlk GRU katmanı
    units1 = hp.Int('units1', 64, 128, step=32)
    model.add(Bidirectional(GRU(units1, return_sequences=True)))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout1', 0.2, 0.4, step=0.1)))

    # İkinci GRU katmanı opsiyonel
    if hp.Boolean('second_gru'):
        units2 = hp.Int('units2', 32, 64, step=16)
        model.add(Bidirectional(GRU(units2)))
        model.add(BatchNormalization())
        model.add(Dropout(hp.Float('dropout2', 0.2, 0.4, step=0.1)))
    else:
        model.add(Bidirectional(GRU(units1)))
        model.add(BatchNormalization())
        model.add(Dropout(hp.Float('dropout2', 0.2, 0.4, step=0.1)))

    # Dense katmanı
    dense_units = hp.Int('dense_units', 32, 64, step=16)
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(hp.Float('dropout3', 0.2, 0.4, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))

    optimizer_choice = hp.Choice('optimizer', ['adam', 'rmsprop'])
    model.compile(optimizer=optimizer_choice, loss='binary_crossentropy', metrics=['accuracy'])

    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=12,
    executions_per_trial=1,
    directory='my_dir',
    project_name='labse_bigru_tuning'
)

tuner.search(
    X_train, y_train,
    epochs=8,
    batch_size=32,
    validation_data=(X_test, y_test)
)

best_model = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best hyperparameters:")
print(f"units1: {best_hp.get('units1')}")
print(f"second_gru: {best_hp.get('second_gru')}")
if best_hp.get('second_gru'):
    print(f"units2: {best_hp.get('units2')}")
print(f"dropout1: {best_hp.get('dropout1')}")
print(f"dropout2: {best_hp.get('dropout2')}")
print(f"dense_units: {best_hp.get('dense_units')}")
print(f"optimizer: {best_hp.get('optimizer')}")

Trial 12 Complete [00h 02m 29s]
val_accuracy: 0.8263134360313416

Best val_accuracy So Far: 0.8292770385742188
Total elapsed time: 00h 29m 15s
Best hyperparameters:
units1: 64
second_gru: False
dropout1: 0.4
dropout2: 0.2
dense_units: 64
optimizer: adam


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
import numpy as np
import pandas as pd

# 1. Tahmin sonuçlarını düzleştir ve int'e çevir
y_pred_flat = y_pred.ravel().astype(int)
y_test = y_test.astype(int)

# 2. Yanlış tahmin edilen indeksleri bul
wrong_indices = np.where(y_pred_flat != y_test)[0]

# 3. Sadece yanlış tahmin edilen yorumları içeren DataFrame
df_wrong = pd.DataFrame({
    "text": [text_test[i] for i in wrong_indices],
    "true_label": [y_test[i] for i in wrong_indices],
    "predicted_label": [y_pred_flat[i] for i in wrong_indices]
})

# 4. Her birini sırayla göster
for idx, row in df_wrong.iterrows():
    print(f"{idx+1}. Gerçek: {row['true_label']} — Tahmin: {row['predicted_label']}")
    print(f"Yorum: {row['text']}\n")

1. Gerçek: 1 — Tahmin: 0
Yorum: Sifariş çox gec gəlir, qiymətlər tez tez dəyişir, məsul dəyişdirmək demə olar ki mümkünsüzdür.

2. Gerçek: 0 — Tahmin: 1
Yorum: Allahakkı qonşumuz Çiçək xalaya belə əhd eləmişdim,həmişə əhdim düşürdu,bu səfər harada səhv elədim bilmirəm

3. Gerçek: 0 — Tahmin: 1
Yorum: Admin, səhvsən, bir gün yaşayıb bioloji düşmənləri tərəfindən ova çevrilən kəpənəyin ömrü bir gündür)

4. Gerçek: 1 — Tahmin: 0
Yorum: Alim olmaq istəyirəm gedim fosillernen qurdalanim maaşımı alımpul cixaranin atasına lenet gece gunduz elleşirik

5. Gerçek: 0 — Tahmin: 1
Yorum: Əla Bütün suallara cavab verir Və Onla Söhbət Etmək olur

6. Gerçek: 1 — Tahmin: 0
Yorum: Mən tv qarşısında oturub çoxda rahat boykotsuz, qadağasız pivəm yanımda futboluma baxacam

7. Gerçek: 1 — Tahmin: 0
Yorum: Çox gözəl proqramdır bəyəndimm

8. Gerçek: 0 — Tahmin: 1
Yorum: Babam heç bir söz demədi, ancaq məni bazara apardı, mənə ayaqqabı aldı, elə o saat da dedi ki, geyin.

9. Gerçek: 0 — Tahmin: 1
Yorum: Bu bar

In [ ]:

for i in wrong_indices:
    text = text_test[i]
    true_label = y_test[i]
    pred_label = y_pred_flat[i]

    print(f"Yorum: {text}")
    print(f"Gerçek etiket: {true_label}, Tahmin: {pred_label}")

    # BURAYA kendi kararını yaz:
    aksiyon = input("Ne yapmak istiyorsun? (s=sil, d=düzelt, g=geç): ").strip().lower()

    if aksiyon == 's':
        df = df[df['text'] != text]  # yorumu sil
        print("❌ Silindi.\n")

    elif aksiyon == 'd':
        df.loc[df['text'] == text, 'labels'] = pred_label  # etiketi tahmine göre düzelt
        print(f"✅ Etiket {pred_label} olarak güncellendi.\n")

    else:
        print("⏩ Geçildi.\n")


Yorum: Salam olsun Mən Batsabda ıstıfabə edə bılmırəm mənə jardim əlını uzadin
Gerçek etiket: 0, Tahmin: 1
Ne yapmak istiyorsun? (s=sil, d=düzelt, g=geç): s
❌ Silindi.

Yorum: Onların dədəsinə rəhmət. Bizim işlətdiylərimizin tərkibi lap betərdir
Gerçek etiket: 0, Tahmin: 1
Ne yapmak istiyorsun? (s=sil, d=düzelt, g=geç): g
⏩ Geçildi.

Yorum: Bir başa catdirma 100 tr olub
Gerçek etiket: 0, Tahmin: 1
Ne yapmak istiyorsun? (s=sil, d=düzelt, g=geç): s
❌ Silindi.

Yorum: pis deyil 01.meri hamı takib atar
Gerçek etiket: 1, Tahmin: 0
Ne yapmak istiyorsun? (s=sil, d=düzelt, g=geç): s
❌ Silindi.

Yorum: Təmiz zibili çıxıb
Gerçek etiket: 1, Tahmin: 0
Ne yapmak istiyorsun? (s=sil, d=düzelt, g=geç): d
✅ Etiket 0 olarak güncellendi.

Yorum: Normal işlədiyi çox az gün var
Gerçek etiket: 0, Tahmin: 1
Ne yapmak istiyorsun? (s=sil, d=düzelt, g=geç): g
⏩ Geçildi.

Yorum: ÇOX baha qiymətə görə
Gerçek etiket: 0, Tahmin: 1
Ne yapmak istiyorsun? (s=sil, d=düzelt, g=geç): g
⏩ Geçildi.

Yorum: Şəkinin havasına

KeyboardInterrupt: Interrupted by user

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sentence_transformers import SentenceTransformer

test_texts = [# Negatif (0)
   "Reklamın reklamı da olar?",
    "Bunu işlətmək üçün diplom lazımdır sanki.",
    "Sadəcə quraşdırdım və peşman oldum.",
    "Gözləyirəm... hələ də gözləyirəm...",
    "Telefonum belə utanır bu tətbiqdən.",
    "Bu nə idi axı? Ciddi-ciddi soruşuram.",
    "2 dəqiqəlik iş üçün 10 səhifə icazə istəyir.",
    "Yenilədim və hər şey daha da pis oldu.",
    "Əsəb pozan dizayn, bir az da səs əlavə edin tam olsun.",
    "Bircə bu çatışmırdı, indi tamam oldu...",
    # Pozitiv (1)
    "Hər şey yerində, heç nə artıq deyil.",
    "Tətbiq mənim əhvalımı düzəldir, ciddi deyirəm.",
    "İlk dəfədir bu qədər rahat hiss edirəm.",
    "İçim rahatdır, nəhayət düzgün bir şey.",
    "Sadə, təmiz və işlək — daha nə olsun?",
    "Sanki beynimi oxuyub hazırlayıblar.",
    "Gecə-gündüz istifadə edirəm, bezmirəm.",
    "Tətbiq deyil, sənət əsəridir bu.",
    "İndi başa düşdüm niyə bu qədər yüklənib.",
    "Heç gözləmirdim, amma möhtəşəmdir."]


# 1. Test cümleleri ve etiketler
test_texts1 = [
    # Pozitiv
    "Bugün özümü çox yaxşı hiss edirəm.",
    "Hər şey planladığım kimi gedir.",
    "Sevdiyim musiqini dinləyəndə ruhum dincəlir.",
    "Səhər günəşi insanın içinə işıq saçır.",
    "Gözlənilmədən dostum zəng etdi, çox sevindim.",
    "Bu gün çox məhsuldar keçdi.",
    "Sadəcə sakitlik və sükunət, başqa heç nə istəmirəm.",
    "Küçədə uşaqların gülüşünü eşitmək çox gözəldir.",
    "Yaxşı insanlar hələ də var, sadəcə tapmaq lazımdır.",
    "Bu xəbəri eşidəndə çox xoşbəxt oldum.",
    "Axşam çayı ilə dincəlmək möhtəşəmdir.",
    "İnsan sevdiklərini yanında görəndə hər şey gözəldir.",
    "Təbiətə çıxmaq mənə çox enerji verir.",
    "Bir anlıq xoşbəxtlik bütün günü gözəl edə bilər.",
    "Gözlədiyim cavab müsbət oldu, şadam.",

    # Neqativ
    "Son vaxtlar heç nəyə həvəsim yoxdur.",
    "İnsanlar çox dəyişib, səmimiyyət azalıb.",
    "Dünən elə yorğun idim ki, heç nə edə bilmədim.",
    "Hər şey üst-üstə gəlir, nəfəs ala bilmirəm.",
    "Bu şəhərdə hər gün eyni darlıq var.",
    "Hər kəs öz işindədir, kimsə maraqlanmır.",
    "İçimdə qəribə bir boşluq var.",
    "Bu qədər səylə etdiyim iş dəyərləndirilmədi.",
    "Günlər keçir, amma heç nə dəyişmir.",
    "Səhərlər durmaq belə istəmirəm.",
    "Bu qədər səssizlik insanı dəli edir.",
    "Artıq heç kimə güvənmirəm.",
    "Yalnızlıq yavaş-yavaş adamı yeyir.",
    "Bəzən hamıdan uzaqlaşmaq istəyirəm.",
    "Həyat sanki yavaşca məni sıxır."
]
test_labels = [0]*10 + [1]*10

# 2. LaBSE modeli ile embedding
labse_model = SentenceTransformer("sentence-transformers/LaBSE")
X_manual = labse_model.encode(test_texts, convert_to_numpy=True)

# 3. Giriş boyutunu BiGRU modeline uygun hale getir (seq_len=1)
X_manual_seq = X_manual.reshape((X_manual.shape[0], 1, X_manual.shape[1]))

# 4. Tahmin yap (olasılık skorları)
pred_probs = model.predict(X_manual_seq).reshape(-1)

# 5. Eşik üstü ise 1, değilse 0 → tahmin etiketi
pred_labels = (pred_probs > 0.5).astype(int)

# 6. Sonuçları DataFrame ile göster
df_manual_test = pd.DataFrame({
    'text': test_texts,
    'true_label': test_labels,
    'predicted_label': pred_labels,
    'score': pred_probs  # burası 0–1 arası olasılık değeridir
})

# 6. Değerlendirme metrikleri
print(classification_report(df_manual_test['true_label'], df_manual_test['predicted_label']))

# 7. Görsel çıktı (Colab uyumlu)
from IPython.display import display
display(df_manual_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
              precision    recall  f1-score   support

           0       0.70      0.70      0.70        10
           1       0.70      0.70      0.70        10

    accuracy                           0.70        20
   macro avg       0.70      0.70      0.70        20
weighted avg       0.70      0.70      0.70        20



,text,true_label,predicted_label,score
0,Reklamın reklamı da olar?,0,0,0.376246
1,Bunu işlətmək üçün diplom lazımdır sanki.,0,0,0.247985
2,Sadəcə quraşdırdım və peşman oldum.,0,0,0.225037
3,Gözləyirəm... hələ də gözləyirəm...,0,1,0.903755
4,Telefonum belə utanır bu tətbiqdən.,0,0,0.182661
5,Bu nə idi axı? Ciddi-ciddi soruşuram.,0,0,0.036697
6,2 dəqiqəlik iş üçün 10 səhifə icazə istəyir.,0,1,0.705859
7,Yenilədim və hər şey daha da pis oldu.,0,0,0.151380
8,"Əsəb pozan dizayn, bir az da səs əlavə edin ta...",0,1,0.515471
9,"Bircə bu çatışmırdı, indi tamam oldu...",0,0,0.283665
